<a href="https://colab.research.google.com/github/Saim-Hassan786/Agentic-AI-With-OpenAI-Agents-SDK/blob/main/10-AgentOutputSchema/AgentOutputSchema.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# AgentOutputSchema
AgentOutputschema is used to handle the output of the SDK produced via LLM to make it same to the type of the output defined in the **output_type** parameter of the Agent class.

In [ ]:
# Installing the SDK
!pip install -Uq openai-agents

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 130.6/130.6 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 129.3/129.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 5.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.2/45.2 kB 1.3 MB/s eta 0:00:00


In [ ]:
# For running event loop
import nest_asyncio
nest_asyncio.apply()

In [ ]:
# Pre requisites SetUp
from google.colab import userdata
GOOGLE_API_KEY= userdata.get('GOOGLE_API_KEY')

from agents import set_default_openai_api,set_default_openai_client,set_tracing_disabled
from openai import AsyncOpenAI

external_client = AsyncOpenAI(
    base_url = "https://generativelanguage.googleapis.com/v1beta/openai/",
    api_key = GOOGLE_API_KEY
)
set_default_openai_client(external_client)
set_default_openai_api("chat_completions")
set_tracing_disabled(True)

# AgentOutputSchemaBase
AgentOutputSchemaBase is the base class for the AgentOutputSchema and implements the following methods in it :

1. **is_plain_text**
2. **json_schema**
3. **is_strict_json_schema**
4. **validate_json**
5. **name**

In [ ]:
# simple output_type code
from agents import Agent,Runner
from pydantic import BaseModel

class OutputType(BaseModel):
    reasoning : str
    response : str
    is_response_correct : bool

agent = Agent(
    name="Simple Assistant ChatBot",
    instructions= "You are a simple assistant that helps user with their queries",
    model = "gemini-2.5-flash",
    output_type = OutputType
)
result = await Runner.run(
    starting_agent=agent,
    input="What are artificial neural networks in 2 lines?",
)
print(result.final_output)
print(result.final_output.reasoning)
print(result.final_output.response)
print(result.final_output.is_response_correct)


reasoning='The response provides a two-line summary defining Artificial Neural Networks.' response='Artificial Neural Networks (ANNs) are computational models inspired by the human brain, designed to recognize patterns and learn from data.\nThey consist of interconnected nodes (neurons) organized in layers that process information and adjust connection strengths (weights) to improve performance.' is_response_correct=True
The response provides a two-line summary defining Artificial Neural Networks.
Artificial Neural Networks (ANNs) are computational models inspired by the human brain, designed to recognize patterns and learn from data.
They consist of interconnected nodes (neurons) organized in layers that process information and adjust connection strengths (weights) to improve performance.
True


In [ ]:
# with AgentOutputSchema
from agents import Agent,Runner,AgentOutputSchema
from pydantic import BaseModel

class OutputType2(BaseModel):
    is_home_work_question : bool
    response : str
    is_response_correct : bool

agent2 = Agent(
    name="Simple Assistant ChatBot",
    instructions= "You are a simple assistant that helps user with their queries",
    model = "gemini-2.5-flash",
    output_type= AgentOutputSchema(output_type=OutputType2,strict_json_schema=True)
)
result2 = await Runner.run(
    starting_agent=agent2,
    input="What are the capitals of France and Germany?",
)
print(result2.final_output)
print(result2.final_output.is_home_work_question)
print(result2.final_output.response)
print(result2.final_output.is_response_correct)

is_home_work_question=False response='The capital of France is Paris and the capital of Germany is Berlin.' is_response_correct=True
False
The capital of France is Paris and the capital of Germany is Berlin.
True


# Parameters of AgentOutputSchema

In [ ]:
json_str = '{"is_home_work_question": true, "response": "Yes, this is homework.", "is_response_correct": false}'
output_schema = AgentOutputSchema(output_type=OutputType2,strict_json_schema=True)
print("Plain Text?", output_schema.is_plain_text())
print("Strict?", output_schema.is_strict_json_schema())
print("Schema:", output_schema.json_schema())
print("Name:", output_schema.name())
print("Validated:", output_schema.validate_json(json_str=json_str))

Plain Text? False
Strict? True
Schema: {'properties': {'is_home_work_question': {'title': 'Is Home Work Question', 'type': 'boolean'}, 'response': {'title': 'Response', 'type': 'string'}, 'is_response_correct': {'title': 'Is Response Correct', 'type': 'boolean'}}, 'required': ['is_home_work_question', 'response', 'is_response_correct'], 'title': 'OutputType2', 'type': 'object', 'additionalProperties': False}
Name: OutputType2
Validated: is_home_work_question=True response='Yes, this is homework.' is_response_correct=False


# **Conclusion**

🟢 **Case 1:** output_type is str or None
🔸 This means the agent expects plain text output (not structured JSON).

🔸 The model's response is treated as a simple string message.

❌ No schema is applied.

❌ No JSON validation is needed.

✅ The response is used directly as-is.



🟡 **Case 2**: output_type is a Pydantic model or dict
✅ A proper JSON schema is generated from the model or dict.

📤 This schema is sent to the LLM (via system prompt/tool call).

🤖 The LLM is instructed to respond in valid JSON matching the schema.

🧪 After the LLM responds, the raw JSON string is:

Validated using the schema (validate_json)

Parsed into the output type (e.g. a BaseModel instance)

✅ You receive a properly validated object that follows your structure.



🔴 **Case 3**: output_type is a primitive (like int, list, float, etc.)
❌ These cannot be described directly using JSON schema alone in a reliable way.

🧊 So they are wrapped in a dict with a special key (e.g. "output").

📤 The SDK constructs a wrapped schema like:

{
  "output": 123  // or a list, or whatever your primitive type is
}

🔄 The LLM is asked to respond in this format.

✅ The wrapper is unwrapped after validation to return the actual value.